In [1]:
import pandas as pd
import os
import numpy as np
import geojson
import cv2
from json.decoder import JSONDecodeError


In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
PATH_root = '/media/user/Mortimer/OV_Verhaak/'
PATH_hovernet = PATH_root+'3_Hovernet/'
PATH_output = '/media/user/Mortimer/OV_Verhaak/4_Athena/1_df_count_cells/'

In [4]:
patients = os.listdir(PATH_hovernet)
print(len(patients))

10


In [5]:
columns_df = {'patient' : [], 'patch' : [], 'cells_tot' : [], 'cells_filtre_bords' : [], 'cells_filtre_diametre' : [], 'cells_filtre_tot' : []}
df = pd.DataFrame.from_dict(columns_df)
df

,patient,patch,cells_tot,cells_filtre_bords,cells_filtre_diametre,cells_filtre_tot


In [6]:
#we remove pixels on the border and we keep only nuclei with an area comprised between 32 and 350 pixels
width = 224
marge = 1
res = 0.23

nbre_pix_inf = 32
nbre_pix_sup = 350

In [7]:
from PIL import Image, ImageDraw
def function_cell(cell) :
    width = 224
    poly_cell = []
    for x in cell["contour"] :
        poly_cell.append((x[0], x[1]))
    width = width
    height = width
    
    mask_cell = np.zeros((width, width), dtype = int)
    
    img = Image.new('L', (width, height), 0)
    ImageDraw.Draw(img).polygon(poly_cell, outline = 1, fill=1)
    mask = np.array(img)
    mask_cell += mask
    
    mask_cell = np.array(mask_cell, dtype = 'uint8')

    
    return mask_cell

In [8]:
def nbre_pixels(cnt1):
    mask_cell = function_cell(cnt1)
    number_of_white_pix = np.sum(mask_cell != 0)
    return number_of_white_pix

In [10]:
for patient in patients :
    if patient not in [''] :
        if not os.path.exists(PATH_output+patient+'.csv'):# and patient not in ['19B2703-V']:
            print(patient)
            PATH_json = PATH_hovernet+patient+'/json/' 
            columns_df = {'patient' : [], 'patch' : [], 'cells_tot' : [], 'cells_filtre_bords' : [], 'cells_filtre_diametre' : [], 'cells_filtre_tot' : []}
            df = pd.DataFrame.from_dict(columns_df)
            jsons = os.listdir(PATH_json)
            print(len(jsons))
            for patch in jsons : 
                if patch not in ['']:#liste:
                    #print(patch)
                    with open(PATH_json+patch) as f :
                        nuclei = geojson.load(f)


                    key_keep = []
                    key_not_keep_bords = []
                    key_not_keep_diametre = []
                    key_not_keep = []

                    cells_tot = len(list(nuclei['nuc'].keys()))
                    
                    for key in list(nuclei['nuc'].keys()) :
                        for x in nuclei['nuc'][key]['contour'] :
                            if x[0] < marge or x[0] >= (width-marge) or x[1] < marge or x[1] >= (width-marge) :
                                key_not_keep_bords.append(key)     
                            pix_count = nbre_pixels(nuclei['nuc'][key])

                            if pix_count < nbre_pix_inf or pix_count > nbre_pix_sup : #
                                key_not_keep_diametre.append(key)
#                                 print('inf', patient, patch, pix_count)
#                                 print('key', key)
#                             elif  pix_count > nbre_pix_sup : #
#                                 key_not_keep_diametre.append(key)
#                                 print('sup', patient, patch, pix_count)
#                                 print('key', key)

                            key_not_keep_bords = list(set(key_not_keep_bords))
                            key_not_keep_diametre = list(set(key_not_keep_diametre))
                            key_not_keep = key_not_keep_bords + key_not_keep_diametre
                            key_not_keep = list(set(key_not_keep))

                        for x in nuclei['nuc'][key]['contour'] :
                            if key not in key_not_keep  :
                                key_keep.append(key) 
                   # print("bzz", len(key_keep))

                    new_row = {'patient' : patient, 'patch' : patch, 'cells_tot' : cells_tot, 'cells_filtre_bords' : cells_tot - len(key_not_keep_bords), 'cells_filtre_diametre' : cells_tot - len(key_not_keep_diametre), 'cells_filtre_tot' : cells_tot - len(key_not_keep)}
                    #df  = df.append(new_row, ignore_index=True)
                    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

                #print(len(df))
            df = df.sort_values(by=['cells_filtre_tot'], ascending = False)
            df.to_csv(PATH_output+patient+'.csv')
        else:
            print(patient, 'déjà traité')

01OV002-467256db-a312-4346-9ac0-e49e44
3712
01OV008-26c6f1bd-0263-4c1f-942e-f1be16
1904
01OV010-a42f5b74-1bab-41cd-b5f7-0182fa
1795
01OV019-2f901c90-617e-46da-9fac-d68369
997
01OV019-c608b23b-dea7-4787-9b64-edd421
2783
01OV029-51a4ce2f-4166-4c6b-a7e0-942d0a
2227
01OV033-ba96e894-15cc-4aed-94ac-b995d7
1184
01OV039-8c7f57cc-aa1c-4e0c-a1cb-0bfb9a
5854
01OV039-8e17b45e-6a06-4e4e-913d-7b1edb
3779
02OV005-40c2f246-6f06-4347-9cc3-7135e0
1114
